In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

c:\Users\zhufe\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\zhufe\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [22]:
class NeuralNetwork(nn.Module):
    """
    A simple neural network with one hidden layer.
    """
    def __init__(self, input_size, hidden_size, output_size, l2_lambda = 0.01):
        super().__init__()

        self.hidden = nn.Linear(input_size, hidden_size)
        nn.init.xavier_uniform_(self.hidden.weight)

        self.bn = nn.BatchNorm1d(hidden_size)

        self.sigmoid = nn.Sigmoid()

        self.dropout = nn.Dropout(p=0.2)

        self.output = nn.Linear(hidden_size, output_size)
        nn.init.xavier_uniform_(self.output.weight)
        
        self.l2_lambda = l2_lambda

    def forward(self, x):
        """
        Performs a forward pass through the neural network.
        """
        x = self.hidden(x)
        x = self.bn(x)
        x = self.sigmoid(x)
        x = self.dropout(x)
        x = self.output(x)
        return x


In [23]:
X = datasets.load_boston()['data']
y = datasets.load_boston()['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
self = NeuralNetwork(input_size = X_train.shape[1], hidden_size = 50, output_size = 1)

In [25]:
criterion = nn.MSELoss()
optimizer = optim.SGD(self.parameters(), 
            lr=0.01, 
            momentum = 0.9, 
            weight_decay=0.01 #l2 reg
            )

In [26]:
self.train()  # turn on dropout

for _ in range(10):
    optimizer.zero_grad()
    output = self(torch.Tensor(X_train))
    loss = criterion(output, torch.Tensor(y_train).view(-1,1))

    # Calculates gradient
    loss.backward()
    print(self.hidden.weight[0])

    # Updates weights
    optimizer.step()

tensor([-0.2589, -0.0437,  0.1635, -0.1984, -0.1975,  0.1917,  0.1334,  0.2452,
        -0.0257, -0.1598, -0.1703, -0.0999,  0.2761], grad_fn=<SelectBackward>)
tensor([-0.2568, -0.0444,  0.1668, -0.1995, -0.1950,  0.1863,  0.1357,  0.2455,
        -0.0231, -0.1568, -0.1674, -0.1021,  0.2822], grad_fn=<SelectBackward>)
tensor([-0.2573, -0.0425,  0.1648, -0.1992, -0.1964,  0.1879,  0.1342,  0.2469,
        -0.0235, -0.1577, -0.1686, -0.1009,  0.2802], grad_fn=<SelectBackward>)
tensor([-0.2639, -0.0318,  0.1506, -0.1898, -0.2083,  0.2150,  0.1228,  0.2512,
        -0.0325, -0.1693, -0.1855, -0.0894,  0.2528], grad_fn=<SelectBackward>)
tensor([-0.2710, -0.0213,  0.1315, -0.1693, -0.2232,  0.2702,  0.1115,  0.2490,
        -0.0430, -0.1859, -0.2178, -0.0678,  0.2033], grad_fn=<SelectBackward>)
tensor([-0.2705, -0.0151,  0.1218, -0.1399, -0.2299,  0.3428,  0.1132,  0.2331,
        -0.0418, -0.1949, -0.2592, -0.0492,  0.1535], grad_fn=<SelectBackward>)
tensor([-0.2590, -0.0125,  0.1210, -0.11

In [27]:
self.eval()  # turn off dropout
with torch.no_grad():  # turn off autograd for faster computation and to save memory
    predicted_output = self(torch.Tensor(X_test))
sum((predicted_output - torch.Tensor(y_test).view(-1,1))**2)

tensor([10584.7480])

In [35]:
for k,v in self.bn.state_dict().items():
    print(f"{k}: {v}")

weight: tensor([0.9032, 1.0739, 0.9745, 1.1197, 0.9367, 1.0587, 0.8422, 0.9181, 1.0094,
        1.0285, 0.9623, 1.0283, 1.0343, 1.0361, 0.8083, 0.9368, 1.0684, 0.9629,
        0.8378, 0.9033, 1.0678, 0.8534, 1.0136, 0.9108, 0.7617, 1.0001, 0.8011,
        0.8332, 1.1556, 0.8562, 1.0705, 0.9595, 0.9404, 1.1152, 0.8646, 1.0941,
        1.1213, 0.9336, 0.7973, 0.8544, 1.1002, 0.9236, 0.8778, 0.9061, 0.9759,
        1.1186, 0.8698, 1.1568, 0.9850, 1.0457])
bias: tensor([-1.0789, -1.1505, -0.9150, -1.0567, -1.0803, -1.0560, -0.7792, -1.0625,
        -0.8704, -1.0204, -0.7998, -0.9115, -0.8365, -0.7837, -0.7820, -1.0993,
        -1.1432, -0.9502, -0.8337, -0.8792, -1.2104, -0.7879, -0.9965, -0.7447,
        -0.7826, -1.1842, -0.8015, -0.8918, -1.1742, -0.8663, -1.2020, -0.9834,
        -0.9962, -1.0520, -1.0658, -1.0654, -1.1532, -1.0108, -1.0254, -0.9121,
        -1.0725, -0.7871, -0.7913, -1.0876, -0.9261, -1.2379, -0.8276, -1.1198,
        -0.9347, -1.0529])
running_mean: tensor([-0.1449,

## QuickProp

In [37]:
import torch
from torch.optim import Optimizer

class QuickProp(Optimizer):
    def __init__(self, params, lr=1e-3):
        if lr < 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))
        defaults = dict(lr=lr)
        super(QuickProp, self).__init__(params, defaults)

    @torch.no_grad()
    def step(self, closure=None):
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for p in group['params']:
                if p.grad is None:
                    continue

                d_p = p.grad
                state = self.state[p]

                # State initialization
                if len(state) == 0:
                    state['prev_delta'] = torch.zeros_like(p, memory_format=torch.preserve_format)
                    state['prev_update'] = torch.zeros_like(p, memory_format=torch.preserve_format)

                prev_delta = state['prev_delta']
                prev_update = state['prev_update']

                denom = prev_delta - d_p + 1e-10  # Add epsilon to prevent division by zero
                update = d_p * prev_update / denom
                p.add_(update, alpha=-group['lr'])

                # Update state
                state['prev_delta'] = d_p.clone()
                state['prev_update'] = update.clone()

        return loss


In [38]:
criterion = nn.MSELoss()
optimizer = QuickProp(self.parameters(), 
            lr=0.01, 
            )

In [39]:
self.train()  # turn on dropout

for _ in range(10):
    optimizer.zero_grad()
    output = self(torch.Tensor(X_train))
    loss = criterion(output, torch.Tensor(y_train).view(-1,1))

    # Calculates gradient
    loss.backward()
    print(self.hidden.weight[0])

    # Updates weights
    optimizer.step()

tensor([-0.2013,  0.0178,  0.0957, -0.0679, -0.2846,  0.6858,  0.1054,  0.1652,
         0.0229, -0.2001, -0.3977,  0.0398, -0.0612], grad_fn=<SelectBackward>)
tensor([-0.2013,  0.0178,  0.0957, -0.0679, -0.2846,  0.6858,  0.1054,  0.1652,
         0.0229, -0.2001, -0.3977,  0.0398, -0.0612], grad_fn=<SelectBackward>)
tensor([-0.2013,  0.0178,  0.0957, -0.0679, -0.2846,  0.6858,  0.1054,  0.1652,
         0.0229, -0.2001, -0.3977,  0.0398, -0.0612], grad_fn=<SelectBackward>)
tensor([-0.2013,  0.0178,  0.0957, -0.0679, -0.2846,  0.6858,  0.1054,  0.1652,
         0.0229, -0.2001, -0.3977,  0.0398, -0.0612], grad_fn=<SelectBackward>)
tensor([-0.2013,  0.0178,  0.0957, -0.0679, -0.2846,  0.6858,  0.1054,  0.1652,
         0.0229, -0.2001, -0.3977,  0.0398, -0.0612], grad_fn=<SelectBackward>)
tensor([-0.2013,  0.0178,  0.0957, -0.0679, -0.2846,  0.6858,  0.1054,  0.1652,
         0.0229, -0.2001, -0.3977,  0.0398, -0.0612], grad_fn=<SelectBackward>)
tensor([-0.2013,  0.0178,  0.0957, -0.06

In [55]:
for k,v in optimizer.state.items():
    print(v)

{'prev_delta': tensor([[ 5.6412e-02, -1.1343e-01,  1.0727e-01,  6.9792e-02,  1.8458e-01,
          1.3629e-01,  1.6069e-01, -5.7747e-02,  1.2965e-01,  1.5276e-01,
         -1.1929e-02, -1.7737e-01,  1.7325e-01],
        [ 5.8839e-01,  6.9606e-02,  3.9354e-01,  4.4279e-01,  4.3578e-01,
         -6.0152e-01,  1.0185e-01,  6.5432e-02,  4.6140e-01,  5.9705e-01,
          3.8745e-01, -3.9846e-01,  7.2509e-01],
        [ 3.6235e-01, -7.8814e-02,  3.8246e-01,  1.3678e-01,  3.4027e-01,
         -5.5767e-01,  2.7598e-01, -7.7951e-02,  3.5379e-01,  4.2411e-01,
          2.6482e-01, -3.0242e-01,  5.5051e-01],
        [ 6.4720e-01,  6.1097e-01,  3.0900e-01,  6.2558e-01,  3.4412e-01,
         -6.3095e-02,  7.4076e-02,  2.1710e-01,  5.2430e-01,  5.8307e-01,
          9.3268e-02, -4.1953e-01,  4.9601e-01],
        [ 6.7004e-02,  8.6858e-02, -3.6196e-02,  3.7515e-02,  5.1374e-02,
          2.4012e-02,  8.8170e-03,  2.4026e-02,  1.2310e-01,  9.4801e-02,
          1.0260e-01, -2.2524e-01,  1.3031e-01],
